In [44]:
from pymodbus.client import ModbusTcpClient
import time
import random

In [45]:
client = ModbusTcpClient('192.168.1.215')
client.connect()

True

In [28]:
def manage_tank_level(tank_value : int, max_tank_value : int):
    meter = "#" * max_tank_value
    print("Current Tank Level", end=" ")
    print("\tMeter")
    print(tank_value, end=" ")
    print("\t\t\t" + meter[0:tank_value])

    coils = client.read_coils(0,6)
    is_tank_full = coils.bits[3]
    is_tank_empty = coils.bits[5]

    if is_tank_empty:
        coils = client.write_coil(2, 1)
    elif is_tank_full:
        coils = client.write_coil(2, 0)
    


Plant Moisture Sensor

In [70]:
plant_moisture = 5
max_moisture = 10
meter = "##########"
dry = False
while True:
    print("Plant Moisture", end=" ")
    print("\tMeter")
    print(plant_moisture, end=" ")
    print("\t\t" + meter[0:plant_moisture])

    coils = client.read_coils(0,6)
    is_pump_on = coils.bits[1]
    is_tank_full = coils.bits[3]
    is_tank_empty = coils.bits[5]

    # used to simulate plant moisture going down
    # at random times between 2 and 10
    random_time = random.randint(2, 10)

    if not is_pump_on:
        plant_moisture -= 1

        if plant_moisture == 0:
            dry = True
    
    if dry and not is_tank_empty:
        plant_moisture += 1
        coils = client.write_coil(0,1)

        if plant_moisture > 8:
            dry = False
            coils = client.write_coil(0,0)

    if plant_moisture > max_moisture:
        plant_moisture = max_moisture
    elif plant_moisture < 0:
        plant_moisture = 0

    if is_pump_on:
        time.sleep(1)
    else:
        time.sleep(random_time)
    
    


Plant Moisture 	Meter
5 		#####
Plant Moisture 	Meter
4 		####
Plant Moisture 	Meter
3 		###
Plant Moisture 	Meter
2 		##
Plant Moisture 	Meter
1 		#
Plant Moisture 	Meter
0 		
Plant Moisture 	Meter
0 		
Plant Moisture 	Meter
1 		#
Plant Moisture 	Meter
1 		#


KeyboardInterrupt: 

In [ ]:

while True:
    holding_registers = client.read_holding_registers(0, 3)
    max_tank_level = holding_registers.registers[0]
    current_tank_level = holding_registers.registers[2]
    manage_tank_level(current_tank_level, max_tank_level)
    time.sleep(1)
    